In [ ]:
# Clone the Darknet repo
!git clone https://github.com/AlexeyAB/darknet

In [ ]:
# Enable GPU and OPENCV
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile

In [ ]:
# Verify CUDA
!/usr/local/cuda/bin/nvcc --version

In [ ]:
# Make the Darknet build
!make

## Define imShow, upload, and download functions

In [5]:
# Function to show predictions
def imShow(path):
  import matplotlib.pyplot as plt
  import cv2
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

# Function for uploading files
def upload():
  from google.colab import files
  uploaded = files.upload() 
  for name, data in uploaded.items():
    with open(name, 'wb') as f:
      f.write(data)
      print ('saved file', name)

# Function for downloading files
def download(path):
  from google.colab import files
  files.download(path)

## Set up the Algorithm

In [ ]:
# Mount Google Drive
%cd ..
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# Makes the path /content/gdrive/My\ Drive/ equal to /mydrive
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

In [ ]:
# Print a list of all files of the MonkeyHealth folder
!ls /content/gdrive/MyDrive/MonkeyHealth

In [9]:
# Make sure we are on the darknet directory
# Sometimes, a folder called "data" will be created outside of the darknet folder when unzipping the obj.zip file
# so we do this to make sure the obj.zip file is unzipped inside the /darknet/data/obj directory
import os
os.chdir('/content/darknet')

In [10]:
# Copy the .zip file into the root directory of cloud VM
!cp /content/gdrive/MyDrive/MonkeyHealth/obj.zip ../

In [ ]:
# Unzip the zip file
!unzip ../obj.zip -d data/

In [12]:
# Upload the custom .cfg file back to cloud VM from Google Drive
!cp /content/gdrive/MyDrive/MonkeyHealth/yolov3_custom.cfg ./cfg

In [13]:
# Upload the obj.names and obj.data files to cloud VM from Google Drive
!cp /content/gdrive/MyDrive/MonkeyHealth/obj.names ./data
!cp /content/gdrive/MyDrive/MonkeyHealth/obj.data  ./data

## Run the Algorithm

In [ ]:
# Set custom .cfg to test mode 
%cd cfg
!sed -i 's/batch=64/batch=1/' yolov3_custom.cfg
!sed -i 's/subdivisions=16/subdivisions=1/' yolov3_custom.cfg
%cd ..

In [ ]:
# Run the object detector based on an image from Google Drive
!./darknet detector test data/obj.data cfg/yolov3_custom.cfg /content/gdrive/MyDrive/MonkeyHealth/backup/yolov3_custom_last.weights /content/gdrive/MyDrive/MonkeyHealth/images/MHTestImage1.jpeg -thresh 0.5 -dont_show
imShow('predictions.jpg')

!cp predictions.jpg /content/gdrive/MyDrive/MonkeyHealth/images/MH1Detection.jpeg